In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
CSV_dataset_URL = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
dataset_df = pd.read_csv(CSV_dataset_URL)
dataset_columns = dataset_df.columns

In [ ]:
print(dataset_df.dtypes)

In [ ]:
number_columns = ['Survived', 'Pclass', 'SibSp', 'Parch', 'Fare']
categorical_columns = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']